# Toolkit Showcase: Prepare and run a protein-ligand simulation

<br />
<details>
    <summary><small>▼ Click here for dependency installation instructions</small></summary>

    The simplest way to install dependencies is to install the examples package:
    
    conda install -c conda-forge openff-toolkit-examples
    
    This example will need access to a GROMACS install in addition to the above dependencies. Your existing GROMACS installed can be used, or you can install it from Bioconda:
    
    conda install -c bioconda gromacs
    
    You can also install all the depencies using the provided environment.yaml:
    
    conda env update --file ../environment.yaml 
    
    You may also need to restart this notebook's kernel after you make these changes (Kernel -> Restart)
</details>


In [1]:
%%html
<!--" Stylesheet for improved readability. This example has a lot of reading, so we want good typography. "-->
<style>
p, ul, ol, h1, h2, h3, h4, h5, h6 {
    max-width: 50rem;    
}
</style>

In [2]:
# Imports from the Python standard library
import sys
from pathlib import Path
from tempfile import NamedTemporaryFile

# Imports from dependencies
from simtk import openmm, unit
import parmed as pmd
import numpy as np
import mdtraj as mdt
import nglview
from openmmforcefields.generators import SMIRNOFFTemplateGenerator

# Imports from the toolkit
import openff.toolkit
from openff.toolkit.typing.engines.smirnoff import ForceField
from openff.toolkit.topology import Molecule, Topology

# Imports from local files
from utils import find_clashing_water, off_topology_to_omm

_(The OpenEye loading warning is expected -- The toolkit is informing us that OETK is unavailable, but it will safely fall back to using RDKit and AmberTools for the same functionality)_

## Introducing the main cast

Merck [provides data](https://github.com/MCompChem/fep-benchmark) to benchmark Free Energy Perturbation (FEP) procedures. We'll use structures from this dataset for this showcase:

https://github.com/MCompChem/fep-benchmark

This example is pre-packaged with one protein-ligand complex from the above repository, however you should be able to download other complexes and run them as well using this workflow. CHEMBL1078774, our ligand of choice, is an inhibitor of the mitotic functions of kinesin-5, a motor protein involved in cell division. 

The ligand and protein structures are already prepared for simulation:

- Their co-ordinates are super-imposable
- Hydrogens added to protein and crystallographic waters
- N-methyl and acetyl terminal caps on the protein to prevent unphysical charges
- Missing atoms are replaced

<br />
<details>
    <summary><small>▼ Click here for the shell commands we used to download the protein-ligand complex</small></summary>

    # Clone the repository
    git clone https://github.com/MCompChem/fep-benchmark.git
    # Take the first ligand from the eg5 benchmark
    head -n119 fep-benchmark/eg5/ligands.sdf > chembl_1078774.sdf
    # Take the prepared protein structure
    cp fep-benchmark/eg5/3l9h_prepared.pdb .
    
</details>

In [3]:
receptor_path = "3l9h_prepared.pdb"
ligand_path = "chembl_1078774.sdf"

We can visualize each structure using the [NGLView](https://github.com/nglviewer/nglview) widget. These visualizations are interactive; rotate by dragging the left mouse button, pan with the right mouse button, and zoom with the scroll wheel. You can also mouse over an atom to see its details, and click an atom to center the view on it. When you mouse over the widget, a full screen button will appear in its top right corner.

In [4]:
view = nglview.show_file(ligand_path)
view

NGLWidget()

In [5]:
view = nglview.show_file(receptor_path)
view

NGLWidget()

# The plan:

| Action | Software|
|--|--|
| Parameterize the ligand | OpenFF Toolkit
| Solvate the protein | OpenMM
| Parameterize the protein | OpenMM
| Combine the ligand and protein into a complex | ParmEd
| Remove waters that clash with the ligand | ParmEd/MDTraj
| Simulate the complex | OpenMM
| Visualize the simulation | NGLView

<div class="alert alert-info" style="max-width: 700px; margin-left: auto; margin-right: auto;">
<!-- TODO: Explain in more detail what OpenMMForceFields, and decide whether to use this in the showcase -->
    ℹ️ Note that there's a new package `OpenMMForceFields` to replace much of this!
    <ul>
        <li> Home: <a href=https://github.com/openmm/openmmforcefields>https://github.com/openmm/openmmforcefields</a> </li>
        <li><code>conda install -c conda-forge openmmforcefields</code></li>
        <li><a href=https://github.com/openforcefield/openforcefield/blob/master/examples/swap_amber_parameters/swap_existing_ligand_parameters_with_openmmforcefields.ipynb>Example notebook available</a></li>
    </ul>
</div>

## Parameterize the ligand (OFF Toolkit)

In this step, we'll produce parameters for our ligand from the unconstrained Parsley 1.3.0 force field. [Parsley](https://openforcefield.org/force-fields/force-fields/#parsley) is the first generation force field produced by the Open Force Field Initiative. Rather than using atom types like traditional biomolecular force fields, Parsley assigns parameters to a molecule with [fancy subgraph matching](https://www.daylight.com/dayhtml/doc/theory/theory.smarts.html). "Unconstrained" denotes that bonds involving hydrogen are treated as harmonics, like other bonds, rather than being constrained to a fixed length. This is appropriate when single-point energies must be as accurate as possible, though constraints allow a greater stable time step.

The Open Force Field Toolkit takes a molecular topology (the atoms in a molecule and their bonds) and the force field specification, and produces a `System` object that can be simulated with OpenMM or converted to inputs for other simulation packages. Note that, to produce a molecular topology, you need more than just the co-ordinates of the atoms; you also need their bonds, bond orders, and formal charges. As a result, `.sdf` files are used in this example; other file types are possible, but they must include this information.

In [6]:
# Load a molecule from a SDF file
ligand = Molecule.from_file(ligand_path)
# Molecule loads both the co-ordinates of the atoms and their bond graph
ligand_positions = ligand.conformers[0]
ligand_topology = ligand.to_topology()

# Load protein and water force field parameters
omm_forcefield = openmm.app.ForceField(
    "amber99sb.xml", "tip3p.xml"
)
# Load the ligand molecule and the Parsley force field
smirnoff = SMIRNOFFTemplateGenerator(forcefield="openff-1.3.0.offxml", molecules=[ligand])
omm_forcefield.registerTemplateGenerator(smirnoff.generator)

In [7]:
# Load the kinesin-5 receptor structure into Modeller
pdb = openmm.app.PDBFile(receptor_path)
modeller = openmm.app.Modeller(pdb.topology, pdb.positions)

# Add the ligand
modeller.add(off_topology_to_omm(ligand_topology), ligand_positions)

# solvate it in 0.15 M NaCl solution
modeller.addSolvent(
    omm_forcefield,
    model="tip3p",
    padding=4.0 * unit.angstrom,
    ionicStrength=0.15 * unit.molar,
)
# Construct the OpenMM System from the solvated structure and the protein force field
system = omm_forcefield.createSystem(
    modeller.topology, 
    nonbondedMethod=openmm.app.PME,
    nonbondedCutoff=9 * unit.angstrom,
    constraints=openmm.app.HBonds,
    rigidWater=True,
)

_(takes ~100 seconds)_
### This is the only block in the first workflow that uses the Open Force Field Toolkit

In this workflow, the Toolkit is just responsible for combining a force field with a molecular topology. It is designed to check the input and give the user useful feedback if there seem to be errors or problems in the provided files. It also computes partial charges without user intervention. Computing charges is a process that can be confusing and error-prone, so we try to specify and simplify it as much as possible. Charges are computed efficiently with [OpenEye](https://www.eyesopen.com/) if is available; if it is not, the free toolkits [RDKit](https://www.rdkit.org/) and [AmberTools](https://ambermd.org/AmberTools.php) are used instead.

    
### Charge generation
* Released FFs only use AM1-BCC, though different semiempirical methods and charge corrections are now available
    * Plans to use other methods in future
* "Graph based" charges are coming in the near future -- Consistency and speed!
    * Currently researching
    * Lets us remove conformer dependence - charges depend on bond graph, not 3D coordinates
    * Potentially a lot faster
* Library charge support is available
    * Template based charges can also be assigned
    

## Solvate and parameterize the protein (OpenMM)

The Parsley force field is designed for small molecule parameterization. Parameters for proteins and other polymers are coming, but for now we'll use Amber 99sb as our protein force field. The protein is already prepared with AMBER-compatible residue names, so this is simple.

<div class="alert alert-warning" style="max-width: 700px; margin-left: auto; margin-right: auto;">
⚠️ Note that OpenMM and the Open Force Field Toolkit both have classes called `Topology` and `ForceField` that serve similar functions. Don't get them confused!
</div>


<div class="alert alert-warning" style="max-width: 700px; margin-left: auto; margin-right: auto;">
⚠️ Note that 4 Å is not enough padding for regular simulations; we're just using it for a fast example.
</div>

## Combine the parameterized ligand and the parameterized protein

[ParmEd](https://github.com/ParmEd/ParmEd) is able to read, write, combine, and edit topologies and structures from several different simulation programs, including CHARMM, OpenMM, Amber, and GROMACS. Here, we use it to combine our protein and ligand systems from OpenMM, remove clashing water molecules, and then write the combined system to OpenMM or GROMACS formats.

In [8]:
# ParmEd can't handle constraints from openmmforcefields, so we need a variant for export without them
export_system = omm_forcefield.createSystem(
    modeller.topology, 
    nonbondedMethod=openmm.app.PME,
    constraints=None,
    rigidWater=False,
)

pmd_complex_struct = pmd.openmm.load_topology(
    modeller.topology, export_system, modeller.positions
)
pmd_complex_struct.box_vectors = modeller.topology.getPeriodicBoxVectors()

## Visualize the complex

NGLView supports a wide variety of [molecular visualization methods](https://nglviewer.org/ngl/api/manual/molecular-representations.html), as well as a VMD-like [atom selection language](https://nglviewer.org/ngl/api/manual/selection-language.html). This can be used to visualize complex systems like this one.

The widget consists of a minimally documented [Python library frontend](https://nglviewer.org/nglview/latest/api.html) and an extensively documented [JavaScript backend](https://nglviewer.org/ngl/api/manual/index.html). You'll need to refer to the documentation for both to do anything sophisticated, as the Python code delegates most of its options and functionality to the JS code.

In [9]:
# Create the widget. By default, proteins are shown as a cartoon and unrecognised ligands with a ball-and-stick model
view = nglview.show_parmed(pmd_complex_struct)
    
# Add a licorice/stick representation for everything except the protein
view.add_licorice(selection="(not protein)")
# Make the ions clearer by drawing their vdW surfaces
view.add_surface(selection=":.NA or :.CL")

# Render the widget
view

NGLWidget()

<div class="alert alert-warning" style="max-width: 700px; margin-left: auto; margin-right: auto;">
⚠️ Note that there are waters clashing with the ligand, since the protein was solvated alone
</div>

## Convert the combined system from ParmEd to our simulation engine of choice

Now that we have our system prepared in ParmEd, we can convert it to the engine we want to run our molecular dynamics simulation on. In this step, we'll use OpenMM; in the next, we'll use GROMACS.

In [10]:
# Construct and configure a Langevin integrator at 300 K with an appropriate friction constant and time-step
integrator = openmm.LangevinIntegrator(
    300 * unit.kelvin, 1 / unit.picosecond, 0.002 * unit.picoseconds
)

# Combine the topology from ParmEd with the new System and integrator
simulation = openmm.app.Simulation(
    pmd_complex_struct.topology, system, integrator
)

# The box is about 75 angstroms per side, so add (30, 30, 30) to center the protein
simulation.context.setPositions(
    pmd_complex_struct.positions + np.array([30, 30, 30]) * unit.angstrom
)

# Add a reporter to record the structure every 10 steps
nc_reporter = pmd.openmm.NetCDFReporter("trajectory.nc", 10)
simulation.reporters.append(nc_reporter)

## Simulate the complex (OpenMM)
### Minimize the combined system

This will reduce any forces that are too large to integrate, such as from clashes or from disagreements between the crystal structure and force field.


In [11]:
simulation.minimizeEnergy()
minimized_state = simulation.context.getState(
    getPositions=True,
    getEnergy=True,
    getForces=True
)

print(
    "Minimised to", 
    minimized_state.getPotentialEnergy(), 
    "with maximum force", 
    max([np.sqrt(v.x*v.x + v.y*v.y + v.z*v.z) for v in minimized_state.getForces()]),
    minimized_state.getForces().unit.get_symbol()
)

minimized_coords = minimized_state.getPositions()

Minimised to -650446.4078558076 kJ/mol with maximum force 2483.0903583040267 kJ/(nm mol)


_(Takes ~2 minutes on single core)_

### Run a short simulation

If this were anything more than a demonstration of the Toolkit, this example would need to include additional steps like equilibration. 

<div class="alert alert-warning" style="max-width: 700px; margin-left: auto; margin-right: auto;">
⚠️ Make sure you use your own, valid simulation protocol! This is just an example.
</div>

In [12]:
simulation.context.setVelocitiesToTemperature(300 * unit.kelvin)
simulation.step(1000)

/home/joshmitchell/conda/envs/offdev/lib/python3.7/site-packages/parmed/amber/netcdffiles.py:409: UserWarning: Could not find netCDF4 module. Falling back on scipy implementation, which can significantly slow down simulations if used as a reporter
  warnings.warn('Could not find netCDF4 module. Falling back on '


_(Takes ~85 seconds on a single core, largely due to trajectory writing frequency)_

## While we wait, a few asides...

<!-- TODO: Get enough extra details to make sense of and expand on these -->

### Force Fields
* Reproducibility - User *must* see the name of what they're using
    * Force field is completely specified by the name, so the name of the force field actually defines all the parameters (cutoff distance, VDW modifiers, constraints, partial charge generation method, etc)
    * As much as possible, we want energy and force to be a deterministic output of combining a molecule and a force field
    * Give the force field name in a paper and it should be reproducible
* Conda data packages - "Plugin" support for additional force fields (anybody can add!)
    * Loads force fields (.offxml) from a list of directories that any package can append to
    * Path to list is in slack
* Evolving together - Toolkit will support all functional forms in [`openff-forcefields` repo](https://github.com/openforcefield/openff-forcefields/) and the SMIRNOFF spec
    * SMIRNOFF is not just a spec, we're also committing to a reference implementation

<img src="img/openforcefields.png" alt="drawing" width="800"/>

<hr/>


### Current cheminformatics toolkit differences - openeye vs rdkit - probs don't need this section
* We try to protect users but there are edge cases
* File formats
* Slight differences in partial charge
* Speed
* SMILES canonicalization
* Behavior stability
* Stereochemistry definition (Edge cases)

## Visualize the simulation (nglview)

NGLView can display trajectories, as well as single structures. Mouse over the widget to see the animation controls.

In [13]:
openmm.app.PDBFile.writeFile(
    pmd_complex_struct.topology, pmd_complex_struct.positions, open("system.pdb", "w")
)
mdt_traj = mdt.load("trajectory.nc", top="system.pdb")
print(mdt_traj)
import nglview

view = nglview.show_mdtraj(mdt_traj)
view

<mdtraj.Trajectory with 100 frames, 40094 atoms, 11914 residues, and unitcells>


NGLWidget(max_frame=99)

## Conclusions
* Toolkit parameterization requires *8 lines*, three of which are cheap hacks 
* Conda-installable, open source tools performed everything from basic system prep to simulation and visualization
* Using OpenMM, we never had to leave Python
* Using ParmEd, there was little additional work to running with GROMACS


<img src="img/dog_food.jpg" alt="drawing" width="350"/>

## What's next?

<!-- TODO: List some examples to continue on with after this one -->

 - Parameter manipulation
 - Conformer energies
 - QCArchive interface

# Appendix: What about GROMACS?

OpenMM makes it easy to run molecular simulations without leaving Python. The OpenFF toolkit currently exports directly to OpenMM, but no part of the parametrization process is exclusively supported by OpenMM. With ParmEd and other tools, the same systems can be run in other engines. Here we show how to use ParmEd to prepare and run the same workflow in GROMACS (_Thanks, Dennis Della Corte!_).

We have [another example](some_url) that uses ParmEd to produce Amber input files too, and also validate single point energies.

<div class="alert alert-warning" style="max-width: 700px; margin-left: auto; margin-right: auto;">
⚠️ Make sure you use your own, valid simulation protocol! This is just an example.
</div>

In [14]:
# Export GROMACS files.
pmd_complex_struct.save("system.top", overwrite=True)
pmd_complex_struct.save("system.gro", overwrite=True)

In [15]:
# TODO: Work out why net charge is  not zero - Seems too big for rounding errors
    # Reason is that the ligand has a charge of 0.004995 according to the topology
    # Known bug: https://github.com/openforcefield/openff-toolkit/issues/456
# TODO: Otherwise clean up the notes produced by GROMACS

# Configure GROMACS not to clutter up our directory with backups
%env GMX_MAXBACKUP=-1

# Energy minimise the coordinates
! gmx grompp -f minim.mdp -c system.gro -p system.top -o em.tpr -maxwarn 1
! gmx mdrun -v -deffnm em


env: GMX_MAXBACKUP=-1
                 :-) GROMACS - gmx grompp, 2021.1-MODIFIED (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meul

Step=   12, Dmax= 7.4e-03 nm, Epot= -4.21756e+05 Fmax= 7.27820e+03, atom= 39760
Step=   13, Dmax= 8.9e-03 nm, Epot= -4.36176e+05 Fmax= 4.28084e+03, atom= 32119
Step=   14, Dmax= 1.1e-02 nm, Epot= -4.53221e+05 Fmax= 3.04210e+03, atom= 40052
Step=   15, Dmax= 1.3e-02 nm, Epot= -4.70334e+05 Fmax= 2.28394e+03, atom= 40052
Step=   16, Dmax= 1.5e-02 nm, Epot= -4.87254e+05 Fmax= 2.03363e+03, atom= 817
Step=   18, Dmax= 9.2e-03 nm, Epot= -4.89652e+05 Fmax= 9.73045e+03, atom= 1607
Step=   19, Dmax= 1.1e-02 nm, Epot= -4.95758e+05 Fmax= 5.53785e+03, atom= 1607
Step=   21, Dmax= 6.7e-03 nm, Epot= -4.98181e+05 Fmax= 3.48515e+03, atom= 1607
Step=   22, Dmax= 8.0e-03 nm, Epot= -4.99719e+05 Fmax= 7.58902e+03, atom= 1607
Step=   23, Dmax= 9.6e-03 nm, Epot= -5.02178e+05 Fmax= 5.38921e+03, atom= 1607
Step=   25, Dmax= 5.8e-03 nm, Epot= -5.04196e+05 Fmax= 2.49241e+03, atom= 1607
Step=   26, Dmax= 6.9e-03 nm, Epot= -5.05882e+05 Fmax= 6.66321e+03, atom= 1607
Step=   27, Dmax= 8.3e-03 nm, Epot= -5.07957e+05 

Step=  143, Dmax= 9.5e-03 nm, Epot= -5.64391e+05 Fmax= 1.05693e+04, atom= 431
Step=  144, Dmax= 1.1e-02 nm, Epot= -5.64726e+05 Fmax= 6.76783e+03, atom= 431
Step=  146, Dmax= 6.8e-03 nm, Epot= -5.64968e+05 Fmax= 3.87407e+03, atom= 431
Step=  147, Dmax= 8.2e-03 nm, Epot= -5.65127e+05 Fmax= 8.41043e+03, atom= 431
Step=  148, Dmax= 9.9e-03 nm, Epot= -5.65370e+05 Fmax= 6.87340e+03, atom= 431
Step=  149, Dmax= 1.2e-02 nm, Epot= -5.65416e+05 Fmax= 1.10019e+04, atom= 431
Step=  150, Dmax= 1.4e-02 nm, Epot= -5.65619e+05 Fmax= 1.09350e+04, atom= 431
Step=  152, Dmax= 8.5e-03 nm, Epot= -5.66029e+05 Fmax= 2.08849e+03, atom= 2031
Step=  153, Dmax= 1.0e-02 nm, Epot= -5.66250e+05 Fmax= 1.37798e+04, atom= 431
Step=  154, Dmax= 1.2e-02 nm, Epot= -5.66794e+05 Fmax= 5.00424e+03, atom= 2031
Step=  156, Dmax= 7.4e-03 nm, Epot= -5.66960e+05 Fmax= 6.37836e+03, atom= 431
Step=  157, Dmax= 8.8e-03 nm, Epot= -5.67122e+05 Fmax= 7.09667e+03, atom= 2031
Step=  158, Dmax= 1.1e-02 nm, Epot= -5.67242e+05 Fmax= 9.1884

Step=  275, Dmax= 1.5e-02 nm, Epot= -5.83530e+05 Fmax= 2.93621e+03, atom= 2031
Step=  276, Dmax= 1.8e-02 nm, Epot= -5.83535e+05 Fmax= 2.39346e+04, atom= 2031
Step=  277, Dmax= 2.1e-02 nm, Epot= -5.83936e+05 Fmax= 8.66194e+03, atom= 2031
Step=  279, Dmax= 1.3e-02 nm, Epot= -5.84004e+05 Fmax= 1.07080e+04, atom= 2031
Step=  280, Dmax= 1.5e-02 nm, Epot= -5.84066e+05 Fmax= 1.26989e+04, atom= 2031
Step=  281, Dmax= 1.8e-02 nm, Epot= -5.84110e+05 Fmax= 1.51902e+04, atom= 2031
Step=  282, Dmax= 2.2e-02 nm, Epot= -5.84130e+05 Fmax= 1.84313e+04, atom= 2031
Step=  283, Dmax= 2.6e-02 nm, Epot= -5.84133e+05 Fmax= 2.16353e+04, atom= 2031
Step=  285, Dmax= 1.6e-02 nm, Epot= -5.84490e+05 Fmax= 2.53701e+03, atom= 2031
Step=  286, Dmax= 1.9e-02 nm, Epot= -5.84510e+05 Fmax= 2.64217e+04, atom= 2031
Step=  287, Dmax= 2.3e-02 nm, Epot= -5.84969e+05 Fmax= 8.70251e+03, atom= 2031
Step=  289, Dmax= 1.4e-02 nm, Epot= -5.85020e+05 Fmax= 1.21437e+04, atom= 2031
Step=  290, Dmax= 1.6e-02 nm, Epot= -5.85090e+05 Fma

Step=  406, Dmax= 1.9e-02 nm, Epot= -5.93622e+05 Fmax= 1.45959e+04, atom= 2031
Step=  408, Dmax= 1.1e-02 nm, Epot= -5.93760e+05 Fmax= 2.67250e+03, atom= 2031
Step=  409, Dmax= 1.3e-02 nm, Epot= -5.93811e+05 Fmax= 1.80419e+04, atom= 2031
Step=  410, Dmax= 1.6e-02 nm, Epot= -5.93989e+05 Fmax= 6.86417e+03, atom= 2031
Step=  412, Dmax= 9.7e-03 nm, Epot= -5.94038e+05 Fmax= 8.07451e+03, atom= 2031
Step=  413, Dmax= 1.2e-02 nm, Epot= -5.94082e+05 Fmax= 9.85892e+03, atom= 2031
Step=  414, Dmax= 1.4e-02 nm, Epot= -5.94121e+05 Fmax= 1.16501e+04, atom= 2031
Step=  415, Dmax= 1.7e-02 nm, Epot= -5.94148e+05 Fmax= 1.41394e+04, atom= 2031
Step=  416, Dmax= 2.0e-02 nm, Epot= -5.94164e+05 Fmax= 1.67947e+04, atom= 2031
Step=  418, Dmax= 1.2e-02 nm, Epot= -5.94336e+05 Fmax= 1.76669e+03, atom= 2031
Step=  419, Dmax= 1.4e-02 nm, Epot= -5.94462e+05 Fmax= 2.04229e+04, atom= 2031
Step=  420, Dmax= 1.7e-02 nm, Epot= -5.94686e+05 Fmax= 6.41046e+03, atom= 2031
Step=  422, Dmax= 1.0e-02 nm, Epot= -5.94724e+05 Fma

Step=  538, Dmax= 1.2e-02 nm, Epot= -6.00125e+05 Fmax= 2.10503e+03, atom= 2031
Step=  539, Dmax= 1.4e-02 nm, Epot= -6.00139e+05 Fmax= 2.01457e+04, atom= 2031
Step=  540, Dmax= 1.7e-02 nm, Epot= -6.00346e+05 Fmax= 6.51547e+03, atom= 2031
Step=  542, Dmax= 1.0e-02 nm, Epot= -6.00372e+05 Fmax= 9.42360e+03, atom= 2031
Step=  543, Dmax= 1.2e-02 nm, Epot= -6.00411e+05 Fmax= 9.70237e+03, atom= 2031
Step=  544, Dmax= 1.5e-02 nm, Epot= -6.00419e+05 Fmax= 1.32234e+04, atom= 2031
Step=  545, Dmax= 1.8e-02 nm, Epot= -6.00445e+05 Fmax= 1.42846e+04, atom= 2031
Step=  547, Dmax= 1.1e-02 nm, Epot= -6.00569e+05 Fmax= 2.21025e+03, atom= 2031
Step=  548, Dmax= 1.3e-02 nm, Epot= -6.00596e+05 Fmax= 1.75691e+04, atom= 2031
Step=  549, Dmax= 1.5e-02 nm, Epot= -6.00758e+05 Fmax= 6.22942e+03, atom= 2031
Step=  551, Dmax= 9.2e-03 nm, Epot= -6.00790e+05 Fmax= 8.04550e+03, atom= 2031
Step=  552, Dmax= 1.1e-02 nm, Epot= -6.00824e+05 Fmax= 9.08411e+03, atom= 2031
Step=  553, Dmax= 1.3e-02 nm, Epot= -6.00844e+05 Fma

Step=  669, Dmax= 1.5e-02 nm, Epot= -6.05697e+05 Fmax= 1.26371e+04, atom= 2031
Step=  671, Dmax= 9.2e-03 nm, Epot= -6.05794e+05 Fmax= 1.50966e+03, atom= 2031
Step=  672, Dmax= 1.1e-02 nm, Epot= -6.05852e+05 Fmax= 1.54266e+04, atom= 2031
Step=  673, Dmax= 1.3e-02 nm, Epot= -6.05979e+05 Fmax= 4.98251e+03, atom= 2031
Step=  675, Dmax= 7.9e-03 nm, Epot= -6.06003e+05 Fmax= 7.26052e+03, atom= 2031
Step=  676, Dmax= 9.5e-03 nm, Epot= -6.06033e+05 Fmax= 7.42548e+03, atom= 2031
Step=  677, Dmax= 1.1e-02 nm, Epot= -6.06045e+05 Fmax= 1.01909e+04, atom= 2031
Step=  678, Dmax= 1.4e-02 nm, Epot= -6.06069e+05 Fmax= 1.09346e+04, atom= 2031
Step=  680, Dmax= 8.2e-03 nm, Epot= -6.06145e+05 Fmax= 1.74725e+03, atom= 2031
Step=  681, Dmax= 9.9e-03 nm, Epot= -6.06189e+05 Fmax= 1.35388e+04, atom= 2031
Step=  682, Dmax= 1.2e-02 nm, Epot= -6.06289e+05 Fmax= 4.76094e+03, atom= 2031
Step=  684, Dmax= 7.1e-03 nm, Epot= -6.06317e+05 Fmax= 6.20619e+03, atom= 2031
Step=  685, Dmax= 8.5e-03 nm, Epot= -6.06345e+05 Fma

Step=  801, Dmax= 9.8e-03 nm, Epot= -6.10191e+05 Fmax= 8.02152e+03, atom= 2031
Step=  802, Dmax= 1.2e-02 nm, Epot= -6.10200e+05 Fmax= 1.00879e+04, atom= 2031
Step=  803, Dmax= 1.4e-02 nm, Epot= -6.10211e+05 Fmax= 1.16427e+04, atom= 2031
Step=  805, Dmax= 8.5e-03 nm, Epot= -6.10292e+05 Fmax= 1.39150e+03, atom= 2031
Step=  806, Dmax= 1.0e-02 nm, Epot= -6.10336e+05 Fmax= 1.42225e+04, atom= 2031
Step=  807, Dmax= 1.2e-02 nm, Epot= -6.10443e+05 Fmax= 4.57560e+03, atom= 2031
Step=  809, Dmax= 7.3e-03 nm, Epot= -6.10462e+05 Fmax= 6.70381e+03, atom= 2031
Step=  810, Dmax= 8.8e-03 nm, Epot= -6.10486e+05 Fmax= 6.82416e+03, atom= 2031
Step=  811, Dmax= 1.1e-02 nm, Epot= -6.10496e+05 Fmax= 9.40708e+03, atom= 2031
Step=  812, Dmax= 1.3e-02 nm, Epot= -6.10516e+05 Fmax= 1.00559e+04, atom= 2031
Step=  814, Dmax= 7.6e-03 nm, Epot= -6.10580e+05 Fmax= 1.62914e+03, atom= 2031
Step=  815, Dmax= 9.1e-03 nm, Epot= -6.10612e+05 Fmax= 1.24402e+04, atom= 2031
Step=  816, Dmax= 1.1e-02 nm, Epot= -6.10696e+05 Fma

Step=  933, Dmax= 6.3e-03 nm, Epot= -6.13910e+05 Fmax= 5.16836e+03, atom= 2031
Step=  934, Dmax= 7.5e-03 nm, Epot= -6.13926e+05 Fmax= 6.42392e+03, atom= 2031
Step=  935, Dmax= 9.0e-03 nm, Epot= -6.13943e+05 Fmax= 7.48542e+03, atom= 2031
Step=  936, Dmax= 1.1e-02 nm, Epot= -6.13953e+05 Fmax= 9.19773e+03, atom= 2031
Step=  937, Dmax= 1.3e-02 nm, Epot= -6.13959e+05 Fmax= 1.08218e+04, atom= 2031
Step=  939, Dmax= 7.8e-03 nm, Epot= -6.14029e+05 Fmax= 1.18671e+03, atom= 2031
Step=  940, Dmax= 9.3e-03 nm, Epot= -6.14073e+05 Fmax= 1.31977e+04, atom= 2031
Step=  941, Dmax= 1.1e-02 nm, Epot= -6.14165e+05 Fmax= 4.11885e+03, atom= 2031
Step=  943, Dmax= 6.7e-03 nm, Epot= -6.14180e+05 Fmax= 6.27224e+03, atom= 2031
Step=  944, Dmax= 8.1e-03 nm, Epot= -6.14202e+05 Fmax= 6.18924e+03, atom= 2031
Step=  945, Dmax= 9.7e-03 nm, Epot= -6.14208e+05 Fmax= 8.76312e+03, atom= 2031
Step=  946, Dmax= 1.2e-02 nm, Epot= -6.14226e+05 Fmax= 9.16758e+03, atom= 2031
Step=  948, Dmax= 7.0e-03 nm, Epot= -6.14279e+05 Fma

Step= 1063, Dmax= 6.7e-03 nm, Epot= -6.16955e+05 Fmax= 1.64989e+03, atom= 2031
Step= 1064, Dmax= 8.0e-03 nm, Epot= -6.16966e+05 Fmax= 1.06928e+04, atom= 2031
Step= 1065, Dmax= 9.6e-03 nm, Epot= -6.17028e+05 Fmax= 4.13487e+03, atom= 2031
Step= 1067, Dmax= 5.8e-03 nm, Epot= -6.17045e+05 Fmax= 4.76127e+03, atom= 2031
Step= 1068, Dmax= 6.9e-03 nm, Epot= -6.17059e+05 Fmax= 5.91752e+03, atom= 2031
Step= 1069, Dmax= 8.3e-03 nm, Epot= -6.17072e+05 Fmax= 6.89365e+03, atom= 2031
Step= 1070, Dmax= 1.0e-02 nm, Epot= -6.17080e+05 Fmax= 8.47575e+03, atom= 2031
Step= 1071, Dmax= 1.2e-02 nm, Epot= -6.17086e+05 Fmax= 9.96553e+03, atom= 2031
Step= 1073, Dmax= 7.2e-03 nm, Epot= -6.17145e+05 Fmax= 1.09733e+03, atom= 2031
Step= 1074, Dmax= 8.6e-03 nm, Epot= -6.17181e+05 Fmax= 1.21499e+04, atom= 2031
Step= 1075, Dmax= 1.0e-02 nm, Epot= -6.17259e+05 Fmax= 3.79961e+03, atom= 2031
Step= 1077, Dmax= 6.2e-03 nm, Epot= -6.17272e+05 Fmax= 5.76889e+03, atom= 2031
Step= 1078, Dmax= 7.4e-03 nm, Epot= -6.17290e+05 Fma

In [16]:
# Run a short simulation
! gmx grompp -f sim.mdp -c em.gro -p system.top -o md.tpr -maxwarn 1
! gmx mdrun -v -deffnm md

                 :-) GROMACS - gmx grompp, 2021.1-MODIFIED (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teem

_(Takes ~100 seconds)_

* Magic:
    * MDP files already prepared
    * `maxwarn 1` becuase of rounding errors with charges

In [19]:
mdt_traj = mdt.load("md.xtc", top="system.gro")
mdt_traj.image_molecules(inplace=True)
view = nglview.show_mdtraj(mdt_traj)
view

NGLWidget(max_frame=10)